In [1]:
import openai
from openai import OpenAI
from dotenv import load_dotenv
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import swifter
from pandas_datareader import data as pdr
import yfinance as yf

/Users/karan_mudaliar/Desktop/CS 7290/Project/KG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

In [3]:
def fetch_description(company_name):
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            max_tokens = 200,
            messages=[
                {"role": "system", "content": "You are a Stock Market Expert that knows a lot about all companies. Provide a brief description of the companies."},
                {"role": "user", "content": f"Describe {company_name}."}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return "Description not available"

In [ ]:
def generate_embeddings(df):
    # Apply the model's encode method to the entire 'About' column for the partition
    model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")
    embeddings = model.encode(df['About'].tolist())
    
    # Store embeddings in a new column. Since embeddings are arrays, you might store them as lists for compatibility
    df['Encoded_About'] = list(embeddings)
    
    return df

In [ ]:
def get_market_cap(ticker_symbol):
    """
    Fetches the market capitalization of a given stock ticker symbol.
    
    Args:
        ticker_symbol (str): The ticker symbol of the stock.
        
    Returns:
        float: The market capitalization of the stock.
    """
    try:
        # Fetching data for the specified stock ticker
        stock = yf.Ticker(ticker_symbol)

        # Getting the market capitalization
        market_cap = stock.info['marketCap']
        
        return market_cap
    except Exception as e:
        print(f"Error occurred: {e}")
        return None